The below cells of code are used to create a Logistic Regression model to predict if a flight out of an aiport with the US will be delayed or not. The first cell of code imports needed packages.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airline_merged.csv',low_memory=False)

In [ ]:
pd.set_option('display.max_columns', None)

After importing the merged file, airline_merged.csv, that was created in the ipynb file, airline_files_merge, we needed to remove the addditional column(Unnamed: 0) that is added in the export. 

In [4]:
#removing unwanted column from previous export
df.drop(columns =['Unnamed: 0'],axis =1)

,year,month,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,...,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,AS,Alaska Airlines Inc.,98,N407AS,ANC,Ted Stevens Anchorage International Airport,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,American Airlines Inc.,2336,N3KUAA,LAX,Los Angeles International Airport,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,US Airways Inc.,840,N171US,SFO,San Francisco International Airport,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,American Airlines Inc.,258,N3HYAA,LAX,Los Angeles International Airport,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,Alaska Airlines Inc.,135,N527AS,SEA,Seattle-Tacoma International Airport,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,JetBlue Airways,688,N657JB,LAX,Los Angeles International Airport,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,JetBlue Airways,745,N828JB,JFK,John F. Kennedy International Airport (New Yor...,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,JetBlue Airways,1503,N913JB,JFK,John F. Kennedy International Airport (New Yor...,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,JetBlue Airways,333,N527JB,MCO,Orlando International Airport,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


The below cell creates a correlation matrix using all of the numerical columns and removes any column that has a correlation coeffecient that is greater than 0.90. This will eventually help decrease the amount of columns that are used in the model and help with any assumptions that need to be met.

In [5]:
#dropping highly correlated columns
#https://www.codegrepper.com/code-examples/python/how+to+drop+highly+correlated+features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df = df.drop(columns = to_drop, axis=1)

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_7999/4014107192.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [6]:
#looking at dataframe after removing highly correlated fields.
df.head()

,Unnamed: 0,year,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,...,taxi_in,scheduled_arrival,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,0,2015,1,4,AS,Alaska Airlines Inc.,98,N407AS,ANC,Ted Stevens Anchorage International Airport,...,4.0,430,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2015,1,4,AA,American Airlines Inc.,2336,N3KUAA,LAX,Los Angeles International Airport,...,4.0,750,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2015,1,4,US,US Airways Inc.,840,N171US,SFO,San Francisco International Airport,...,11.0,806,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2015,1,4,AA,American Airlines Inc.,258,N3HYAA,LAX,Los Angeles International Airport,...,8.0,805,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2015,1,4,AS,Alaska Airlines Inc.,135,N527AS,SEA,Seattle-Tacoma International Airport,...,5.0,320,0,0,NaN,NaN,NaN,NaN,NaN,NaN


Since the data set is quite large and personal laptops will struggle to churn through the amount of rows and columns, we decided to filter down the data to Dallas-Fort Worth International Airport. This decreased the data frame size from almost 6 million rows to just under 240k rows and also helped with the one hot encoding below.

In [7]:
#selecting 5 busiest airports in US
#airports = ['ATL','DFW','ORD','LAX','DEN']
airports = ['DFW']

In [8]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [9]:
df_air.describe()

,Unnamed: 0,year,day,day_of_week,flight_number,latitude_origin,latitude_destination,scheduled_departure,departure_delay,taxi_out,...,wheels_on,taxi_in,scheduled_arrival,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,2.395510e+05,239551.0,239551.000000,239551.000000,239551.000000,0.0,0.0,239551.000000,233658.000000,233383.000000,...,233177.000000,233177.000000,239551.000000,239551.000000,239551.000000,50478.000000,50478.000000,50478.000000,50478.000000,50478.000000
mean,2.683140e+06,2015.0,15.644890,3.945598,1999.527111,NaN,NaN,1406.443325,11.528820,17.242344,...,1556.667347,6.466109,1579.647649,0.002713,0.026107,10.543623,0.113000,21.174888,20.654245,5.074389
std,1.672708e+06,0.0,8.784501,1.996657,1155.423485,NaN,NaN,458.043842,36.308279,8.543331,...,507.568388,5.213076,487.907272,0.052020,0.159454,22.054115,2.274275,39.279341,39.726000,24.350348
min,7.000000e+01,2015.0,1.000000,1.000000,4.000000,NaN,NaN,140.000000,-28.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.243666e+06,2015.0,8.000000,2.000000,1164.000000,NaN,NaN,1025.000000,-4.000000,13.000000,...,1155.000000,4.000000,1206.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.577904e+06,2015.0,16.000000,4.000000,2214.000000,NaN,NaN,1400.000000,-1.000000,15.000000,...,1555.000000,5.000000,1604.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000
75%,3.935769e+06,2015.0,23.000000,6.000000,2810.000000,NaN,NaN,1820.000000,11.000000,19.000000,...,1950.000000,7.000000,1959.000000,0.000000,0.000000,15.000000,0.000000,27.000000,26.000000,0.000000
max,5.819003e+06,2015.0,31.000000,7.000000,6529.000000,NaN,NaN,2359.000000,1377.000000,225.000000,...,2400.000000,197.000000,2359.000000,1.000000,1.000000,664.000000,190.000000,942.000000,1331.000000,862.000000


In [10]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

Because we want to predict if an airplane is going to be delayed or not using logistic regression, we created a new field called "is_delay" from the "departure_delay" field. To do this, we created a function that we could pass a numeric field through and if the value for that row is greater than 0, we imputed "1" into the curated field to signifiy the flight was delayed. If the value was less than 0, we imputed "0" to signify that the flight was not delayed. 

In [11]:
#create binary response for logistic regression
# create a function
def is_delayed(delay,axis=1):
    if delay <= 0:
        return 0
    else:
        return 1
    
# create a new column based on condition
df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_7999/3939397851.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)


After creating the "is_delay" field, we then began to One Hot Encode any categorial variables so that our model could interpret and use those columns. This caused a 250k by 42 data set to become 240k by 3400. 

In [12]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on non-numeric column s
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [13]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

#head(final_df)

Once the target variable was created and we one hot encoded the data set, we were able to begin modeling our data. To start, we separated the features with the target varibale as to not leave the response in a test or train data set.

In [14]:
feature_cols = final_df.loc[:, final_df.columns != 'is_delay'].columns
X = final_df[feature_cols] # Features
y = final_df.is_delay # Target variable

From sklearn, we used their train_test_split package to create a 75/25 train and test split for our model.

In [15]:
#split dataset in features and target variable
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

After one hot encoding, for our logistic regession model to run, we imputed the NaN values with 0.

In [16]:
#imputing 0 into NaN values for logistic regression after OHE
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

In [17]:
#checking shape of data
print('X_train shape: ',X_train.shape,'\n')
print('X_test shape: ',X_test.shape,'\n')
print('y_train shape: ',y_train.shape,'\n')
print('y_test shape: ',y_test.shape)

X_train shape:  (179663, 3395) 

X_test shape:  (59888, 3395) 

y_train shape:  (179663,) 

y_test shape:  (59888,)


#CV implement::::
#
# Create an instance of StratifiedKFold which can be used to get indices of different training and test folds
#
strtfdKFold = StratifiedKFold(n_splits=10)
kfold = strtfdKFold.split(X, y)
scores = []
#
#
#
for k, (train, test) in enumerate(kfold):
    pipe.fit(X_train.iloc[train, :], y_train.iloc[train])
    score = pipe.score(X_train.iloc[test, :], y_train.iloc[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, Accuracy: %.3f' % (k+1, np.bincount(y_train.iloc[train]), score))
 
print('\n\nCross-Validation accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

To simplify the code and useability of the model, we used a pipeline function from sklearn to standardize (using StandarScaler), run the data through PCA and then finally a logistic regession model with a CV = 10. 

In [20]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

# Scale -> PCA -> LR w/ CV=10
logistic = LogisticRegressionCV(solver = 'saga',penalty='elasticnet',l1_ratios = 0.5,cv=10 )
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("logistic", logistic)])
pipe.fit(X_train,y_train)

TypeError: object of type 'float' has no len()

After fitting the model, we then ran our test data set through to create predictions.

In [ ]:
#predict
y_pred=pipe.predict(X_test)

After creating predictions on our test data set, we pulled metrics to determine how well our model performed. Our base model had an accuracy of 78.38% with a precision of 96.08% and a recall of 51.54%. This model output gives us a good starting point for hypertuing the model and comparison against some more powerful models such as XGBoost or LightGBM. After computing these metrics we then created a confusion matrix heatmap to better visualize how well our model was predicting and where the model could perform better with False Positives and False Negatives.

In [ ]:
#Metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
#correlation Matrix Heatmap
# code from: https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
cf_matrix = confusion_matrix(y_test, y_pred)

group_names = ['True Neg','False Pos','False Neg','True Pos']

group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

Finally we chose to plot a ROC curve to visualize specificity and sensitiviy of our model. 

In [ ]:
#ROC Curve
y_pred_proba = pipe.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="Base Logistic Regression, auc= "+str(auc))
plt.legend(loc=4)
plt.show()